In [1]:
from keras import applications, optimizers
from keras.layers import Dropout, Flatten, Dense, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import keras
import numpy as np

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = '../dataset/train'
test_data_dir = '../dataset/test'
validation_data_dir = '../dataset/validation'
nb_train_samples = 1792
nb_test_samples = 210
nb_validation_samples = 525

In [9]:
def buildModel():
    # build the ResNET50 network
    base_model = applications.ResNet50(weights='imagenet', 
                                    include_top=False,
                                    input_tensor=Input(shape=(img_width, img_height, 3)))

    for layer in base_model.layers:
        layer.trainable = False
        
    top_model = base_model.output
    top_model = Flatten(name="Flatten")(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(256, activation='relu')(top_model)
    top_model = Dense(6, activation='softmax')(top_model)
    
    model = Model(inputs=base_model.input, outputs=top_model)

    # model.summary()
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    return model

In [4]:
def generateData(batch_size):
    train_datagen = ImageDataGenerator(
        horizontal_flip=True,
        rescale=1. /255,
        rotation_range=30,
        shear_range=0.2,
        zoom_range=0.2)

    validation_datagen = ImageDataGenerator(rescale=1. /255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')
    
    return [train_generator,validation_generator]

In [5]:
def fineTuneModel(model, train_generator, validation_generator, epochs, batch_size):
    model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)
    
    return model

In [6]:
def getMetrics(model, batch_size):
    
    test_datagen = ImageDataGenerator(rescale=1. /255)

    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width,img_height),
        shuffle=False,
        batch_size=1,
        class_mode='categorical')
    
    predicted_results = model.predict_generator(test_generator, 
                                                steps = nb_test_samples)
    predicted_results = np.argmax(predicted_results, axis=1)
    targets = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]
    
    # confusion matrix
    print("CONFUSION MATRIX:")
    print(confusion_matrix(test_generator.classes, predicted_results))
    
    # classification report
    print("CLASSIFICATION REPORT:")
    print(classification_report(test_generator.classes, predicted_results, target_names=targets))

In [11]:
def runExperiments():
    batch_sizes = [8,16,32]
    epochs = [10,20,30]
    config = 1
    
    for batch_size in batch_sizes:
        for epoch in epochs:
            
            print("*************** Test ", config, " Batch size: ", batch_size, " Epochs: ", epoch, "***************")           
            keras.backend.clear_session()
            
            model = buildModel()
            train_generator,test_generator = generateData(batch_size)
            trained_model = fineTuneModel(model,train_generator, test_generator, epoch, batch_size)
            metrics = getMetrics(trained_model, batch_size)
            
            config += 1

In [ ]:
runExperiments()

*************** Test  1  Batch size:  8  Epochs:  10 ***************
Found 1792 images belonging to 6 classes.
Found 525 images belonging to 6 classes.
Epoch 1/10
224/224 [==============================] - 585s 3s/step - loss: 12.8501 - acc: 0.1998 - val_loss: 13.0185 - val_acc: 0.1923
Epoch 2/10
224/224 [==============================] - 560s 3s/step - loss: 12.8801 - acc: 0.2009 - val_loss: 13.0628 - val_acc: 0.1896
Epoch 3/10
224/224 [==============================] - 561s 3s/step - loss: 12.8801 - acc: 0.2009 - val_loss: 12.8446 - val_acc: 0.2031
Epoch 4/10
224/224 [==============================] - 561s 3s/step - loss: 12.8801 - acc: 0.2009 - val_loss: 13.0628 - val_acc: 0.1896
Epoch 5/10
224/224 [==============================] - 557s 2s/step - loss: 12.8801 - acc: 0.2009 - val_loss: 13.0940 - val_acc: 0.1876
Epoch 6/10
224/224 [==============================] - 555s 2s/step - loss: 12.8801 - acc: 0.2009 - val_loss: 12.9381 - val_acc: 0.1973
Epoch 7/10
224/224 [==================

/home/martin/anaconda3/envs/martin/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Found 1792 images belonging to 6 classes.
Found 525 images belonging to 6 classes.
Epoch 1/20
224/224 [==============================] - 586s 3s/step - loss: 11.9940 - acc: 0.2517 - val_loss: 13.0495 - val_acc: 0.1904
Epoch 2/20
224/224 [==============================] - 563s 3s/step - loss: 12.0346 - acc: 0.2533 - val_loss: 13.0005 - val_acc: 0.1934
Epoch 3/20
224/224 [==============================] - 563s 3s/step - loss: 12.0346 - acc: 0.2533 - val_loss: 13.1875 - val_acc: 0.1818
Epoch 4/20
224/224 [==============================] - 563s 3s/step - loss: 12.0346 - acc: 0.2533 - val_loss: 12.9693 - val_acc: 0.1954
Epoch 5/20
224/224 [==============================] - 564s 3s/step - loss: 12.0346 - acc: 0.2533 - val_loss: 13.1252 - val_acc: 0.1857
Epoch 6/20
224/224 [==============================] - 561s 3s/step - loss: 12.0346 - acc: 0.2533 - val_loss: 13.0005 - val_acc: 0.1934
Epoch 7/20
224/224 [==============================] - 562s 3s/step - loss: 12.0346 - acc: 0.2533 - val_loss

Found 1792 images belonging to 6 classes.
Found 525 images belonging to 6 classes.
Epoch 1/10
112/112 [==============================] - 558s 5s/step - loss: 12.9717 - acc: 0.1881 - val_loss: 13.0330 - val_acc: 0.1914
Epoch 2/10
112/112 [==============================] - 514s 5s/step - loss: 13.0420 - acc: 0.1908 - val_loss: 12.9515 - val_acc: 0.1965
Epoch 3/10
112/112 [==============================] - 512s 5s/step - loss: 13.0420 - acc: 0.1908 - val_loss: 13.2048 - val_acc: 0.1807
Epoch 4/10
112/112 [==============================] - 512s 5s/step - loss: 13.0420 - acc: 0.1908 - val_loss: 12.9515 - val_acc: 0.1965
Epoch 5/10
112/112 [==============================] - 511s 5s/step - loss: 13.0420 - acc: 0.1908 - val_loss: 13.0781 - val_acc: 0.1886
Epoch 6/10
112/112 [==============================] - 510s 5s/step - loss: 13.0420 - acc: 0.1908 - val_loss: 13.2365 - val_acc: 0.1788
Epoch 7/10
112/112 [==============================] - 510s 5s/step - loss: 13.0420 - acc: 0.1908 - val_loss

In [12]:
runExperiments()

*************** Test  8  Batch size:  32  Epochs:  20 ***************
Found 1792 images belonging to 6 classes.
Found 525 images belonging to 6 classes.
Epoch 1/20
56/56 [==============================] - 526s 9s/step - loss: 10.8637 - acc: 0.2997 - val_loss: 12.9808 - val_acc: 0.1934
Epoch 2/20
56/56 [==============================] - 479s 9s/step - loss: 10.3266 - acc: 0.3555 - val_loss: 12.9468 - val_acc: 0.1968
Epoch 3/20
56/56 [==============================] - 483s 9s/step - loss: 10.9366 - acc: 0.3198 - val_loss: 13.1103 - val_acc: 0.1866
Epoch 4/20
56/56 [==============================] - 481s 9s/step - loss: 10.9635 - acc: 0.3186 - val_loss: 13.0122 - val_acc: 0.1927
Epoch 5/20
56/56 [==============================] - 481s 9s/step - loss: 12.8621 - acc: 0.2020 - val_loss: 13.0449 - val_acc: 0.1907
Epoch 6/20
56/56 [==============================] - 479s 9s/step - loss: 12.7958 - acc: 0.2059 - val_loss: 13.0122 - val_acc: 0.1927
Epoch 7/20
56/56 [==============================]